In [ ]:
import cv2

In [ ]:
img = cv2.imread('love.jpg',1)

In [ ]:
print(type(img))

In [ ]:
print(img.shape)

In [ ]:
resized = cv2.resize(img,(600,600))

In [ ]:
cv2.imshow('Legend',resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
face_cascade = cv2.CascadeClassifier('C:\\Users\\dell\\Anaconda\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
gray_img = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)

In [ ]:
faces = face_cascade.detectMultiScale(gray_img,scaleFactor=1.3,minNeighbors=5)

In [ ]:
print(type(faces))
print(faces)

In [ ]:
for x,y,w,h in faces:
    resized = cv2.rectangle(resized,(x,y),(x+w,y+h),(0,255,0),3)

In [ ]:
cv2.imshow("Gray",resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

# video capture

In [ ]:
import cv2
video = cv2.VideoCapture(0)
check,frame=video.read()
print(check)
print(frame)

In [ ]:
import time
time.sleep(3)
cv2.imshow('Capture',frame)
cv2.waitKey(0)

video.release()
cv2.destroyAllWindows()

# Video Capturing

In [ ]:
import cv2,time
video = cv2.VideoCapture(0)
a=1

In [ ]:
while True:
    a=a+1
    check, frame=video.read()
    print(frame)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    cv2.imshow('Capture',gray)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
print(a)
video.release()
cv2.destroyAllWindows()

# Motion Detector

In [1]:
import cv2,time,pandas
from datetime import datetime

In [6]:
first_frame=None
status_list=[None,None]
times=[]
df = pandas.DataFrame(columns=['Start','End'])


In [7]:
video = cv2.VideoCapture(0)
while True:
    check,frame=video.read()
    status = 0
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(21,21),0)
    
    if first_frame is None:
        first_frame=gray
        continue
    
    
    
    delta_frame=cv2.absdiff(first_frame,gray)
    thresh_delta = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta,None,iterations=0)
    (_,cnts,_) = cv2.findContours(thresh_delta.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for contour in cnts:
        if cv2.contourArea(contour)<5000:
            continue
        status=1
        (x,y,w,h) = cv2.boundingRect(contour)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
    status_list.append(status)
    status_list=status_list[-2:]
    
    if status_list[-1]==1 and status_list[-2]==0:
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now())
        
    cv2.imshow('frame',frame)
    cv2.imshow('Capturing',gray)
    cv2.imshow('delta',delta_frame)
    cv2.imshow('thresh',thresh_delta)
    key = cv2.waitKey(1)
    if key==ord('q'):
        if status==1:
            times.append(datetime.now())
        break
print(status_list)
print(times)
for i in range(0,len(times),2):
    df = df.append({'Start':times[i],'End':times[i+1]},ignore_index  =True)
df.to_csv('Times.csv')
video.release()
cv2.destroyAllWindows()

[0, 0]
[datetime.datetime(2019, 1, 19, 15, 37, 29, 403213), datetime.datetime(2019, 1, 19, 15, 37, 30, 698751), datetime.datetime(2019, 1, 19, 15, 37, 31, 899541), datetime.datetime(2019, 1, 19, 15, 37, 33, 419480), datetime.datetime(2019, 1, 19, 15, 37, 33, 899198), datetime.datetime(2019, 1, 19, 15, 37, 35, 52116), datetime.datetime(2019, 1, 19, 15, 37, 36, 827372), datetime.datetime(2019, 1, 19, 15, 37, 37, 821715), datetime.datetime(2019, 1, 19, 15, 37, 38, 635540), datetime.datetime(2019, 1, 19, 15, 37, 45, 710632), datetime.datetime(2019, 1, 19, 15, 37, 47, 35092), datetime.datetime(2019, 1, 19, 15, 37, 48, 13478)]


In [8]:
from bokeh.plotting import figure, show, output_file, Figure
from bokeh.models import HoverTool, ColumnDataSource
df['Start'] = pandas.to_datetime(df['Start'])
df['End'] = pandas.to_datetime(df['End'])
df['Start_string']=df['Start'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['End_string']=df['End'].dt.strftime('%Y-%m-%d %H:%M:%S')
cds = ColumnDataSource(df)
p=figure(x_axis_type = 'datetime',height=100,width=500,responsive=True,title='Motion Graph')
p.yaxis.minor_tick_line_color=None

C:\Users\dell\Anaconda\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: responsive parameter was deprecated in Bokeh 0.12.10 and will be removed, use sizing_mode='fixed' for responsive=False or sizing_mode='scale_width' for responsive=True instead.
  warn(message)


In [9]:
p.ygrid[0].ticker.desired_num_ticks=1

hover=HoverTool(tooltips=[("Start","@Start_string"),("End","@End_string")])
p.add_tools(hover)

q=p.quad(left="Start",right="End",bottom=0,top=1,color="green",source=cds)

output_file("Graph1.html")
show(p)